In [1]:
import neo4j
import pandas as pd

In [2]:
belong_to_dept_df = pd.read_csv("belong_to_dept.csv")
has_action_df = pd.read_csv("has_action.csv")
has_case_df = pd.read_csv("has_case.csv")
seek_approval_df = pd.read_csv("seek_approval.csv")

In [4]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
user = "neo4j"
password = "jeremiahtay"
gds = GraphDataScience("bolt://localhost:7687", auth=(user,password))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'


In [5]:
# Q1: Using Neo4j Aura, create the Knowledge Graph below
create_starting_knowledge_graph = """
CREATE (d:Department {name:"Department"})
CREATE (a:Approver {name:"Approver"})
CREATE (p:Person {name: "Person"})
CREATE (c:Case {case: "Case"})
CREATE (x:Action {action: "Action"})

CREATE (p)-[:belongtoDept]->(d)
CREATE (p)-[:seekApproval]->(a)
CREATE (p)-[:hasCase]->(c)
CREATE (c)-[:hasAction]->(x)
"""

In [6]:
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response



conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="neo4j",              
                       pwd="jeremiahtay")

# Load Departments and Persons
query_departments_persons = """
LOAD CSV WITH HEADERS FROM 'file:///belong_to_dept.csv' AS row
MERGE (p:Person {name: row.Person, role: row.Role})
MERGE (d:Department {name: row.Department})
MERGE (p)-[:belongtoDept]->(d);
"""

# Load Approvals
query_approvals = """
LOAD CSV WITH HEADERS FROM 'file:///seek_approval.csv' AS row
MATCH (p:Person {name: row.Person})
MATCH (a:Person {name: row.Approver})
MERGE (p)-[:seekApproval]->(a);
"""

# Load Cases
query_cases = """
LOAD CSV WITH HEADERS FROM 'file:///has_case.csv' AS row
MATCH (p:Person {name: row.Person})
MERGE (c:Case {id: toInteger(row.CaseID), description: row.CaseDescription})
MERGE (p)-[:hasCase]->(c);
"""

# Load Actions
query_actions = """
LOAD CSV WITH HEADERS FROM 'file:///has_action.csv' AS row
MATCH (c:Case {id: toInteger(row.CaseID)})
MERGE (x:Action {id: toInteger(row.CaseID), action: row.Action, status: row.Status})
MERGE (c)-[:hasAction]->(x);
"""

conn.query(query_departments_persons)
conn.query(query_approvals)
conn.query(query_cases)
conn.query(query_actions)




[]

In [14]:
# Scenario 1 Q1a: How many people are there in LOB department?
num_lob = ("""
MATCH p = (a:Person)-[r:belongtoDept]-> (d:Department {name: "LOB"})
RETURN COUNT(p) AS Num_LOB_Employee
""")
print(conn.query(num_lob))

# Scenario Q1b: How many case load are there for HR department?
num_case_load_HR = ("""
MATCH (a:Person)-[r:belongtoDept]->(d:Department {name: "HR"})
MATCH (a)-[r1:hasCase]->(c:Case)
MATCH (c)-[r2:hasAction]->(x:Action)
RETURN COUNT(a) AS Num_HR_Case_Load
""")
print(conn.query(num_case_load_HR))

[<Record LOB_Employees=2>]
[<Record HR_case_load=3>]


In [7]:
# Scenario 2a: Using the above Knowledge Graph and dataset, Can you infer that Rachel’s approving officer is Gerard.
uri="bolt://localhost:7687"
ser="neo4j"      
pwd="jeremiahtay"

def establish_seek_approval(person_a, person_b):
    # Initiate connection to the database
    connection = Neo4jConnection(uri, user, pwd)
    
    parameters = {
        "person_a_name": person_a["name"],
        "person_a_role": person_a["role"],
        "person_b_name": person_b["name"],
        "person_b_role": person_b["role"]
    }

    # Checking to ensure that two team members will not seek approval from each other
    if person_a["role"] == "Team Member" and person_b["role"] == "Team Member":
        print(f"{person_a['name']} has not established a seekApproval relationship with {person_b['name']}")
        return
    
    # Checking for any current Team Member that seeks approval from any person with the role 'Head'
    check_any_team_member_query = """
    MATCH (p1:Person {role: 'Team Member'})-[:seekApproval]->(head:Person {role: 'Head'})-[:belongtoDept]->(d:Department {name: 'HR'}),
          (p1)-[:belongtoDept]->(d),
          (p2:Person {name: $person_a_name, role: $person_a_role})-[:belongtoDept]->(d)
    RETURN p1, head, p2
    """

    team_member_nodes = connection.query(check_any_team_member_query, parameters)

    if team_member_nodes:
        # Forming any possible seekApproval relationship between person_a and person_b
        add_relationship_query = """
        MATCH (p3:Person {name: $person_a_name, role: $person_a_role})-[:belongtoDept]->(department:Department {name: 'HR'}),
              (p4:Person {name: $person_b_name, role: $person_b_role})-[:belongtoDept]->(department)
        MERGE (p3)-[:seekApproval]->(p4)
        RETURN p3, p4
        """
        relationship_result = connection.query(add_relationship_query, parameters)
        print(f"{person_a['name']} has established a seekApproval relationship with {person_b['name']}")
    else:
        print(f"{person_a['name']} has not established a seekApproval relationship with {person_b['name']}")

# Execute the function
person_a_seeking_approval = {"name": "Rachel", "role": "Team Member"}
person_b_approver = {"name": "Gerard", "role": "Head"}
establish_seek_approval(person_a_seeking_approval, person_b_approver)


Rachel has established a seekApproval relationship with Gerard


In [19]:
# Challenge Scenario – (no coding required) Describe how you can predict that most of the time, HR’s Action is to ‘Find Resource’. 
# What Knowledge Graph techniques can be used? Please elaborate.
"""
Answer:
Applying graph analytics:
Firstly, I would identify clusters in the current knowledge graph database by querying a relationship link from each department to the actions of the case. 
Next, I would compute  the aggrigated probability, that represents how often "Find Resource" occurs relative to the total number of cases managed by the HR department.
If the probability is high (setting a threshold for what constitutes a high probability), it would confirm that "Find Resource" is the most common action favoured by the HR Department.
"""

'\nAnswer:\nFirstly, I would use the current knowledge graph database to query a relationship link between each department and the cases\' actions. \nNext, I would calculate the aggrigated probability, which is calculating the number of cases being "Find Resource" over the total number of cases handled by the HR department.\n\n'